Введение в анализ данных (первый семестр)
Домашнее задание №3 - Участие в конкурсе Rossmann Store Sales
Дедлайн сдачи домашнего задания: 28 апреля, 23:59. После дедлайна решения не принимаются.

В данном домашнем задании вам придется учавствовать в конкурсе Rossman Store Sales. Ваша задача получить наиболее высокий результат на Private Leaderboard, используя только те методы, которые вы узнали до сегодняшнего дня на курсах Техносферы. Оценка домашнего задания будет производиться через kaggle kernels. До дедлайна вам необходимо прислать на почту письмо с ссылкой на ваш kernel. Не забудьте указать правильно тему:)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

<h2>Подготовка</h2>
Импорт всех необходимых пакетов:

In [ ]:
import gc
import operator
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit
import matplotlib.pyplot as plt
%matplotlib inline

Описание некоторых необходимых функций, используемых далее:

In [ ]:
def to_weight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(y, yhat):
    w = to_weight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe

def str_date_to_ordinal(date):
    return datetime.strptime(date, '%Y-%m-%d').date().toordinal()

def date_to_ordinal(date):
    return date.toordinal()

def str_to_date(date):
    return datetime.strptime(date, '%Y-%m-%d').date()

SEED = 42

<h2>Загрузка и предобработка данных</h2>
<h3>Загрузка тренировочных и тестовых данных</h3>

In [ ]:
train_df = pd.read_csv('../input/train.csv', sep=',', parse_dates=['Date'], date_parser=str_to_date,
                      low_memory=False)
test_df = pd.read_csv('../input/test.csv', sep=',', parse_dates=['Date'], date_parser=str_to_date,
                      low_memory=False)

<h3>Обработка пропущенных значений в тренировочных и тестовых данных</h3>

In [ ]:
train_df.info()

Как видно из информации о тренировочной выборке, пропущенных значений в ней нет.

In [ ]:
test_df.info()

Как видно из информации о тестовой выборке, пропущенных значений в ней нет нигде, кроме 11 пропущенных значений переменной Open.

In [ ]:
test_df[pd.isnull(test_df.Open)]

Видно, что все 11 пропущенных значений относятся к магазину 622. Предположим, что магазин был открыт (Open == 1), т.к. если окажется, что магазин был закрыт, то мы предскажем Sales = 0, и это никак не повлияет на ошибку в предсказании. В случае же, если магазин был на самом деле открыт, а мы предположили, что он закрыт и автоматически предсказали 0, мы получим ошибку.

In [ ]:
test_df.loc[pd.isnull(test_df.Open), 'Open'] = 1

<h3>Обработка категориальных признаков в тренировочных и тестовых данных</h3>
Объединим выборки в одну для анализа значений категориальных переменных. Заметим, что в тестовой выборке есть переменная Id, не принимающая NaN значений, а в тренировочной выборке данной переменной нет. Поэтому по значению Nan или non-NaN данной переменной будем различать тренировочную и тестовую выборки.

In [ ]:
all_df = pd.concat([train_df, test_df], axis=0)
all_df.head()

In [ ]:
all_df.info()

In [ ]:
unique_StateHoliday_values = np.sort(all_df.StateHoliday.unique())
print ("Train data StateHoliday unique values:", np.sort(train_df.StateHoliday.unique()))
print ("Test data StateHoliday unique values: ", np.sort(test_df.StateHoliday.unique()))
print ("All data StateHoliday unique values:  ", unique_StateHoliday_values)

Пронумеруем все уникальные значения переменной StateHoliday: 

In [ ]:
all_df['StateHoliday'] = all_df['StateHoliday'].astype('category').cat.codes

Преобразуем переменную Date.

In [ ]:
feature_name = 'Date'
dispatcher = {'DayOfMonth': pd.Index(all_df[feature_name]).day, 
              'WeekOfYear': pd.Index(all_df[feature_name]).week,
              'MonthOfYear': pd.Index(all_df[feature_name]).month,
              'Year': pd.Index(all_df[feature_name]).year,
              'DayOfYear': pd.Index(all_df[feature_name]).dayofyear
             }

for new_feat_suffx, mapping in dispatcher.items():
    all_df[feature_name + new_feat_suffx] = mapping

all_df[feature_name] = all_df[feature_name].apply(date_to_ordinal)

all_df.head()

<h3>Загрузка данных о магазинах</h3>

In [ ]:
store_df = pd.read_csv('../input/store.csv', sep=',')
store_df.head()

In [ ]:
store_df.info()

Видно, что дистанции до магазинов-конкурентов определены практически для всех магазинов, а это значит, что даты открытия конкурентов не могут иметь какие-либо краевые значения (т.к. конкуренты действительно у этих магазинов есть). Поэтому в будущем, при преобразовании данных признаков в дату, будем пропущенные значения заполнять средним значением.

<h3>Обработка категориальных признаков в данных о магазинах</h3>

Категориальные признаки - StoreType и Assortment. Также обработаем все временные признаки.

In [ ]:
unique_StoreType_values =np.sort(store_df['StoreType'].unique())
unique_Assortment_values = np.sort(store_df['Assortment'].unique())
print ("Unique values of StoreType: ", unique_StoreType_values)
print ("Unique values of Assortment:", unique_Assortment_values)

In [ ]:
store_df['StoreType'] = store_df['StoreType'].astype('category').cat.codes
store_df['Assortment'] = store_df['Assortment'].astype('category').cat.codes
print ("Unique values of StoreType: ", np.sort(store_df['StoreType'].unique()))
print ("Unique values of Assortment:", np.sort(store_df['Assortment'].unique()))

Далее необходимо преобразовать переменные CompetitionOpenSinceYear и CompetitionOpenSinceMonth к числовому представлению, чтобы можно было сравнивать для каждой из покупок, была ли она совершена в данный период, или нет. Пропуски заполним, как уже говорилось, средними значениями.

In [ ]:
def CompetitionOpenSince_to_ordinal(row):
    try:
        date = '%d-%d' % (int(row['CompetitionOpenSinceYear']), int(row['CompetitionOpenSinceMonth']))
        return datetime.strptime(date, '%Y-%m').date().toordinal()
    except:
        return np.nan

In [ ]:
store_df['CompetitionOpenSinceDate'] = store_df.apply(CompetitionOpenSince_to_ordinal, axis=1)
mean_CompetitionOpenSince = store_df['CompetitionOpenSinceDate'].mean()
store_df['CompetitionOpenSinceDate'] = store_df['CompetitionOpenSinceDate'].fillna(
                                        mean_CompetitionOpenSince).astype(np.int64)
store_df.head()

Теперь необходимо преобразовать переменные Promo2SinceYear и Promo2SinceWeek числовому представлению, чтобы можно было сравнивать для каждой из покупок, была ли она совершена в данный период, или нет. Для этого введём переменную Promo2SinceDate. Если Promo2SinceYear и Promo2SinceWeek заданы для магазина, то Promo2SinceDate = date(Promo2SinceYear, Promo2SinceWeek).toordinal(), иначе  Promo2SinceDate = 0 для магазинов с Promo2 == 0, Promo2SinceDate = среднему для магазинов с Promo2 == 1.

In [ ]:
def Promo2Since_to_ordinal(row):
    try:
        date = '%d-W%d' % (int(row['Promo2SinceYear']), int(row['Promo2SinceWeek']))
        return datetime.strptime(date + '-1', '%Y-W%W-%w').date().toordinal()
    except:
        return np.nan

In [ ]:
store_df['Promo2SinceDate'] = store_df.apply(Promo2Since_to_ordinal, axis=1)
mean_Promo2SinceDate = store_df['Promo2SinceDate'].mean()
store_df['Promo2SinceDate'] = store_df['Promo2SinceDate'].fillna(mean_Promo2SinceDate).astype(np.int64)
store_df.loc[store_df['Promo2'] == 0, 'Promo2SinceDate'] = 0
store_df.head()

И, наконец, необходимо преобразовать переменную PromoInterval: будем хранить кортеж номеров месяцев,  которые проходят сезонные распродажи, либо мода, если значение пропущенное.

In [ ]:
mode_PromoInterval = store_df[store_df['PromoInterval'].notnull()]['PromoInterval'].mode()[0]
promo_intervals = {'Jan,Apr,Jul,Oct': (1,4,7,10), 
                   'Feb,May,Aug,Nov': (2,5,8,11), 
                   'Mar,Jun,Sept,Dec': (3,6,9,12)
                  } # ()}
promo_intervals[np.nan] = promo_intervals[mode_PromoInterval]
store_df['PromoInterval'] = store_df['PromoInterval'].apply(lambda x: promo_intervals[x])
store_df.head()

<h3>Удаление ненужных признаков из данных о магазинах</h3>

In [ ]:
columns_to_drop = ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',
                   'Promo2SinceWeek', 'Promo2SinceYear']
store_df.drop(columns_to_drop, axis=1, inplace=True)
store_df.head()

<h3>Обработка пропущенных значений в данных о магазинах</h3>

In [ ]:
store_df.info()

Пропущенные значения для признака CompetitionDistance заполним средним значением:

In [ ]:
mean_CompetitionDistance = store_df.CompetitionDistance.mean()
store_df['CompetitionDistance'] = store_df['CompetitionDistance'].fillna(mean_CompetitionDistance)
store_df.info()

<h3>Слияние данных о продажах и о магазинах</h3>

In [ ]:
df = pd.merge(all_df, store_df, how='left', on=['Store'])
df.head().T

In [ ]:
df.info()

<h3>Введение новых признаков для учёта промоакций и конкурентов</h3>

Введём 3 новых переменных:<br>
1) Promo2Today - есть ли сегодня сезонная распродажа в этом магазине<br>
2) CompetitionIsOpen - открыт ли сегодня какой-либо конкурент этого магазина<br>
3) AverageCheck - средний чек для этого магазина за всё время<br>

In [ ]:
def is_promo2_today(row):
    return int(row['Promo2'] == 1 and row['Promo2SinceDate'] <= row['Date'] \
           and row['DateMonthOfYear'] in row['PromoInterval'])

def is_competition_open(row):
    return int(row['CompetitionOpenSinceDate'] <= row['Date'])

In [ ]:
df['Promo2Today'] = df.apply(is_promo2_today, axis=1)
df['CompetitionIsOpen'] = df.apply(is_competition_open, axis=1)
avg_checks = pd.DataFrame(df.groupby('Store')['Sales'].sum().astype(np.float64) \
             / df.groupby('Store')['Customers'].sum().astype(np.float64), 
                          columns=['AverageCheck']).reset_index()
df = df.merge(avg_checks, on='Store', how='left')
df.head().T

Удаление столбцов, не нужных для применения модели:

In [ ]:
columns_to_drop = ['PromoInterval', 'Date', 'CompetitionOpenSinceDate', 'Promo2SinceDate']
df.drop(columns_to_drop, axis=1, inplace=True)

<h2>Применение Random Forest Regressor для предсказания продаж</h2>
Будем анализировать только открытые магазины (закрытый магазин должен приносить прибыль 0):

In [ ]:
train_df = df[(df['Id'].isnull()) & (df['Open'] == 1)].drop(['Id'], axis=1)
train_df.info()

In [ ]:
test_df = df[df['Id'].notnull()].drop(['Sales', 'Customers'], axis=1)
test_df.info()

In [ ]:
X_train = train_df[train_df.columns.drop(['Customers', 'Sales'])].values
y_train = train_df['Sales'].values
print (X_train.shape, y_train.shape)

In [ ]:
X_test = test_df[test_df.columns.drop(['Id'])].values
X_test.shape

Выбираем тренировочную и тестовую подвыборки небольшого размера:

In [ ]:
#X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(X_train, y_train,
                                                                           #train_size=0.08,
                                                                           #test_size=0.02,
                                                                           #random_state=SEED)
#print X_train_train.shape, X_train_test.shape, y_train_train.shape, y_train_test.shape

In [ ]:
columns = train_df.columns.drop(['Sales', 'Customers'])
del train_df
gc.collect()

Подбираем наилучшие параметры модели с помощью поиска по сетке параметров с использованием кросс-валидации на 5 разбиениях временного ряда:

In [ ]:
'''
param_grid = {'n_estimators': (10, 50, 80, 100),
              'criterion': ('mse',),
              'max_depth': (5, 10, 15, 20, None)}
best_n_estimators = None
best_criterion = None
best_max_depth = None
best_rsmpe_score = 100.0
for criterion in param_grid['criterion']:
    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            print ('n_estimators =', n_estimators, 'criterion =', criterion, 'max_depth =', max_depth)
            scores = []
            tss = TimeSeriesSplit(n_splits=5)
            for train, test in tss.split(X_train_train):
                rf_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,
                                                 criterion=criterion, random_state=SEED, n_jobs=1)
                rf_model.fit(X_train_train[train], y_train_train[train])
                scores.append(rmspe(y_train_train[test], rf_model.predict(X_train_train[test])))
                del rf_model
                gc.collect()
            cur_rsmpe_score = np.mean(np.array(scores))
            print ('rsmpe_score =', cur_rsmpe_score)
            if cur_rsmpe_score < best_rsmpe_score:
                best_rsmpe_score = cur_rsmpe_score
                best_n_estimators = n_estimators
                best_criterion = criterion
                best_max_depth = max_depth
            del scores, tss
            gc.collect()
            print '----------------------------------------------------------'
        
print ('best n_estimators:', best_n_estimators, 'best criterion:', best_criterion)
print ('rmspe for best params:', best_rsmpe_score)
'''

n_estimators = 10 criterion = mse max_depth = 5
rsmpe_score = 0.444912158098
----------------------------------------------------------
n_estimators = 10 criterion = mse max_depth = 10
rsmpe_score = 0.380676726535
----------------------------------------------------------
n_estimators = 10 criterion = mse max_depth = 15
rsmpe_score = 0.296350019077
----------------------------------------------------------
n_estimators = 10 criterion = mse max_depth = 20
rsmpe_score = 0.239696414616
----------------------------------------------------------
n_estimators = 10 criterion = mse max_depth = None
rsmpe_score = 0.220703127173
----------------------------------------------------------
n_estimators = 50 criterion = mse max_depth = 5
rsmpe_score = 0.444539510519
----------------------------------------------------------
n_estimators = 50 criterion = mse max_depth = 10
rsmpe_score = 0.378287301831
----------------------------------------------------------
n_estimators = 50 criterion = mse max_depth = 15
rsmpe_score = 0.290571644102
----------------------------------------------------------
n_estimators = 50 criterion = mse max_depth = 20
rsmpe_score = 0.228885966621
----------------------------------------------------------
n_estimators = 50 criterion = mse max_depth = None
rsmpe_score = 0.20681522393
----------------------------------------------------------
n_estimators = 80 criterion = mse max_depth = 5
rsmpe_score = 0.444897937191
----------------------------------------------------------
n_estimators = 80 criterion = mse max_depth = 10
rsmpe_score = 0.378541263317
----------------------------------------------------------
n_estimators = 80 criterion = mse max_depth = 15
rsmpe_score = 0.290696131506
----------------------------------------------------------
n_estimators = 80 criterion = mse max_depth = 20
rsmpe_score = 0.22799154794
----------------------------------------------------------
n_estimators = 80 criterion = mse max_depth = None
rsmpe_score = 0.205834885351
----------------------------------------------------------
n_estimators = 100 criterion = mse max_depth = 5
rsmpe_score = 0.444946184302
----------------------------------------------------------
n_estimators = 100 criterion = mse max_depth = 10
rsmpe_score = 0.378800830837
----------------------------------------------------------
n_estimators = 100 criterion = mse max_depth = 15
rsmpe_score = 0.290867452618
----------------------------------------------------------
n_estimators = 100 criterion = mse max_depth = 20
rsmpe_score = 0.22810305889
----------------------------------------------------------
n_estimators = 100 criterion = mse max_depth = None
rsmpe_score = 0.205413980062
----------------------------------------------------------
best n_estimators: 100 best criterion: mse
rmspe for best params: 0.205413980062


best n_estimators: 100<br>
best max_depth: None

Обучим модель с наилучшими найденными параметрами на всех тестовых данных, взяв количество деревьев равным 50, т.к. оценка для 50 деревьев почти не отличается от оценки для 100, а время обучения модели для 50 деревьев меньше:

In [ ]:
rf = RandomForestRegressor(n_estimators=50, random_state=SEED)
rf.fit(X_train, y_train)

In [ ]:
# check RMSPE on test data to verify regression is not too bad
#print rmspe(y_train_test, rf.predict(X_train_test))

In [ ]:
test_predicted = rf.predict(X_test)
test_predicted.shape

In [ ]:
submission = pd.concat([test_df.Id.astype(int), pd.DataFrame(test_predicted, 
                                                 columns=['Sales'], index=test_df.index)], axis=1)
submission.head()

In [ ]:
submission[['Id', 'Sales']].to_csv('submission_6.csv', index=False)

Посмотрим важность признаков:

In [ ]:
features = {}
for i, column in enumerate(columns):
    features[column] = rf.feature_importances_[i] 
features = sorted(features.items(), key=operator.itemgetter(1), reverse=True)
features

Лучший результат:<br>
Private Score 0.15533<br>
Public Score 0.13161